In [1]:
from glob import glob
import json
from tqdm import tqdm

In [2]:
selected = [
    '/home/husein/ssd3/instructions/synthetic-codealpaca-v1-chatgpt4.jsonl',
    '/home/husein/ssd3/instructions/synthetic-glaive_coder_raw_text.jsonl',
    '/home/husein/ssd3/instructions/synthetic-oss_instruct-decontaminated.jsonl'
]

In [3]:
rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'jawaban',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas',
    'pertama-tama',
    'bagian',
    'fungsionalitas',
    'komoditas',
    'diekspor',
    ' akkan',
    ' bisa',
    'keren',
    'konfigurkan',
    'communique',
    'berbeda',
    'bagian-bagian',
    'butuh',
    'bahwa',
    ' membaut'
]

indons = []
def found_word(s, words):
    global indons
    for i in range(len(words)):
        if words[i] in s:
            indons.append(s)
            return True
    return False

In [4]:
def replace_code(s):
    s = s.replace(' Kode ', ' Kod ')
    s = s.replace(' kode ', ' kod ')
    s = s.replace('Kode ', 'Kod ')
    s = s.replace('kodenya', 'kodnya')
    s = s.replace('Kodenya', 'Kodnya')
    s = s.replace('opkode', 'opkod')
    s = s.replace(' kode', ' kod')
    s = s.replace('dikodekan', 'dikodkan')
    s = s.replace('kodks', 'kodeks')
    s = s.replace('kode ', 'kod ')
    s = s.replace(' Kode', ' Kod')
    return s

In [5]:
english_news = glob('/home/husein/ssd3/google-translate-english-news/*.requested')
len(english_news)

32

In [6]:
english_texts = glob('/home/husein/ssd3/google-translate-english-texts/*.requested')
len(english_texts)

39

In [7]:
malay_news = glob('/home/husein/ssd3/google-translate-malay-news/*.requested')
len(malay_news)

22

In [8]:
hansard = glob('/home/husein/ssd3/google-translate-malaysia-hansard/*.requested')
len(hansard)

5

In [9]:
ens = [english_news, english_texts]
mss = [malay_news, hansard]

In [10]:
def reject(k):
    if 'return JSON' in k:
        return
    if 'AI language model' in k:
        return
    if 'model bahasa AI' in k:
        return
    return True

In [11]:
folders = [
    'zh-CN', 'ta', 'jw', 'id'
]

sublanguages = []
for f in folders:
    sublanguages.extend(glob(f'/home/husein/ssd3/google-translate-ms-{f}/ms*.requested'))
    
len(sublanguages)

69

In [12]:
with open('train.json', 'w') as fopen_jsonl:

    with open('/home/husein/ssd3/llama2/postfilter.jsonl') as fopen:
        for l in fopen:
            l = json.loads(l)

            en = l['instruction']
            ms = l['instruction_ms']

            if not en or not ms:
                continue

            if not reject(en) or not reject(ms):
                continue

            ms = replace_code(ms)

            if found_word(ms, rejected_words):
                continue

            if len(en) and len(ms):
                
                s = f'Malay to English: {ms} {en}'
                    
                fopen_jsonl.write(f'{json.dumps(s)}\n')
                fopen_jsonl.flush()
                
                s = f'English to Malay: {en} {ms}'
                    
                fopen_jsonl.write(f'{json.dumps(s)}\n')
                fopen_jsonl.flush()

In [13]:
with open('train.json', 'a') as fopen_jsonl:
    
    for f in selected:
        with open(f) as fopen:
            for l in fopen:
                l = json.loads(l)
                en = l['instruction_en']
                ms = l['instruction_ms']

                if not en or not ms:
                    continue

                if not reject(en) or not reject(ms):
                    continue
                
                ms = replace_code(ms)
        
                if found_word(ms, rejected_words):
                    continue
                
                if len(en) and len(ms):
                    s = f'Malay to English: {ms} {en}'
                    
                    fopen_jsonl.write(f'{json.dumps(s)}\n')
                    fopen_jsonl.flush()

                    s = f'English to Malay: {en} {ms}'

                    fopen_jsonl.write(f'{json.dumps(s)}\n')
                    fopen_jsonl.flush()

In [14]:
with open('train.json', 'a') as fopen_jsonl:
    with open('/home/husein/ssd3/chatgpt4-noisy-translation-twitter-dialect/negeri-kelantan.jsonl') as fopen:
        for l in fopen:
            l = json.loads(l)
            left = l['r']['translation']
            right = l['original']['ms']
            
            if found_word(right, rejected_words):
                continue
            
            if len(left) and len(left.split()) < 2000 and len(right) and len(right.split()) < 2000:

                s = f'Kelantanese to Standard Malay: {left} {right}'
                    
                fopen_jsonl.write(f'{json.dumps(s)}\n')
                fopen_jsonl.flush()
            
    with open('/home/husein/ssd3/chatgpt4-noisy-translation-twitter-dialect/negeri-utara.jsonl') as fopen:
        for l in fopen:
            l = json.loads(l)
            left = l['r']['translation']
            right = l['original']['ms']
            
            if found_word(right, rejected_words):
                continue
            
            if len(left) and len(left.split()) < 2000 and len(right) and len(right.split()) < 2000:

                s = f'Northern language to Standard Malay: {left} {right}'
                    
                fopen_jsonl.write(f'{json.dumps(s)}\n')
                fopen_jsonl.flush()

In [15]:
from huggingface_hub import hf_hub_download
manglish = hf_hub_download(
    repo_id="mesolitica/chatgpt-noisy-translation-manglish", 
    filename="process-manglish.jsonl", repo_type = 'dataset'
)

In [16]:
with open('train.json', 'a') as fopen_jsonl:
    with open(manglish) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            left = l['left']
            en = l['en']
            ms = l['ms']
            
            if not reject(en) or not reject(ms):
                continue

            if len(left) and len(left.split()) < 2000 and len(en) and len(en.split()) < 2000:
                
                s = f'Manglish to Standard English: {left} {en}'
                    
                fopen_jsonl.write(f'{json.dumps(s)}\n')
                fopen_jsonl.flush()
                
                s = f'Standard English to Manglish: {en} {left}'
                    
                fopen_jsonl.write(f'{json.dumps(s)}\n')
                fopen_jsonl.flush()
                
            if found_word(ms, rejected_words):
                continue
                
            if len(left) and len(left.split()) < 2000 and len(ms) and len(ms.split()) < 2000:
                
                s = f'Manglish to Standard Malay: {left} {ms}'
                    
                fopen_jsonl.write(f'{json.dumps(s)}\n')
                fopen_jsonl.flush()
                
                s = f'Standard Malay to Manglish: {ms} {left}'
                    
                fopen_jsonl.write(f'{json.dumps(s)}\n')
                fopen_jsonl.flush()

1036411it [00:18, 55162.55it/s]


In [17]:
bjn = hf_hub_download(
    repo_id="mesolitica/chatgpt-noisy-translation-banjarese", 
    filename="process-bjn-Latn.jsonl", repo_type = 'dataset'
)

In [18]:
with open('train.json', 'a') as fopen_jsonl:
    with open(bjn) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            left = l['left']
            chatgpt = l['chatgpt']
            google = l['google']
            
            if not reject(chatgpt) or not reject(google):
                continue
                
            if found_word(google, rejected_words):
                continue
            
            if len(left) and len(left.split()) < 2000 and len(google) and len(google.split()) < 2000:
                
                s = f'Banjarese to Malay: {left} {google}'
                    
                fopen_jsonl.write(f'{json.dumps(s)}\n')
                fopen_jsonl.flush()

958405it [00:14, 64478.33it/s]


In [19]:
fb = hf_hub_download(
    repo_id="mesolitica/chatgpt-noisy-translation-facebook", 
    filename="processed-facebook.jsonl", repo_type = 'dataset'
)
twitter = hf_hub_download(
    repo_id="mesolitica/chatgpt-noisy-translation-twitter", 
    filename="processed-twitter.jsonl", repo_type = 'dataset'
)
iium = hf_hub_download(
    repo_id="mesolitica/chatgpt-noisy-translation-iium-confession", 
    filename="processed-iium-confession.jsonl", repo_type = 'dataset'
)
bcari = hf_hub_download(
    repo_id="mesolitica/chatgpt-noisy-translation-b.cari.com.my", 
    filename="processed-b.cari.com.my.jsonl", repo_type = 'dataset'
)
ccari = hf_hub_download(
    repo_id="mesolitica/chatgpt-noisy-translation-c.cari.com.my", 
    filename="processed-c.cari.com.my.jsonl", repo_type = 'dataset'
)

In [20]:
with open('train.json', 'a') as fopen_jsonl:
    for f in [
        twitter, 
        fb, 
        iium,
        bcari,
    ]:
        with open(f) as fopen:
            for l in tqdm(fopen):
                try:
                    l = json.loads(l)
                    left = l['left']
                    en = l['en']
                    ms = l['ms']
                    
                    if not reject(en) or not reject(ms):
                        continue

                    if len(left) and len(left.split()) < 2000 and len(en) and len(en.split()) < 2000:
                        s = f'social media Malay to standard English: {left} {en}'
                    
                        fopen_jsonl.write(f'{json.dumps(s)}\n')
                        fopen_jsonl.flush()
                        
                    if found_word(ms, rejected_words):
                        continue

                    if len(left) and len(left.split()) < 2000 and len(ms) and len(ms.split()) < 2000:
                        s = f'social media Malay to standard Malay: {left} {ms}'
                    
                        fopen_jsonl.write(f'{json.dumps(s)}\n')
                        fopen_jsonl.flush()
                        
                except Exception as e:
                    pass

691013it [00:09, 75032.24it/s]
141851it [00:01, 85171.61it/s]
333758it [00:08, 40754.50it/s]
631727it [00:11, 53557.44it/s]


In [21]:
!tail -n 1 train.json

"social media Malay to standard Malay: Hadoi. Tapi okay la ni at least cina vs cina. Kalau cina vs melayu or cina vs india kang kata racist pulak. Hadoi. Tapi tak apa la, setidaknya ini pertembungan antara orang Cina dengan orang Cina. Kalau pertembungan antara orang Cina dengan orang Melayu atau orang Cina dengan orang India, orang akan kata itu rasis."


In [22]:
with open('train.json', 'a') as fopen_jsonl:
    for f in [
        ccari,
    ]:
        with open(f) as fopen:
            for l in tqdm(fopen):
                try:
                    l = json.loads(l)
                    left = l['left']
                    en = l['en']
                    ms = l['ms']
                    
                    if not reject(en) or not reject(ms):
                        continue

                    if len(left) and len(left.split()) < 2000 and len(en) and len(en.split()) < 2000:
                        s = f'social media Mandarin to standard English: {left} {en}'
                    
                        fopen_jsonl.write(f'{json.dumps(s)}\n')
                        fopen_jsonl.flush()
                        
                    if found_word(ms, rejected_words):
                        continue

                    if len(left) and len(left.split()) < 2000 and len(ms) and len(ms.split()) < 2000:
                        s = f'social media Mandarin to standard Malay: {left} {ms}'
                    
                        fopen_jsonl.write(f'{json.dumps(s)}\n')
                        fopen_jsonl.flush()
                        
                except Exception as e:
                    pass

740431it [00:26, 28466.65it/s]


In [23]:
!tail -n 1 train.json

"social media Mandarin to standard Malay: \u56e0\u4e3a\u6211\u6709\u4e2afree room\u6240\u4ee5\u6253\u7b97\u53ea\u662f\u53bb\u4e2ashort trip\u90a3\u6837\u800c\u5df2  \u8001\u516c\u53c8\u4e0d\u80fd\u62ff\u592a\u591a\u5047\u671f\u3002  \u4e00\u5929\u7684\u8bdd\u6211\u89c9\u5f97\u53ef\u4ee5\u53bbcable car+sky bridge (sky bridge\u4f1a\u722c\u5230\u8981\u6b7b\uff0c\u4e0d\u597d\u8fd0\u52a8\u7684\u8bdd\u5c31\u4e0d\u8981\u53bb\u4e86\uff09  \u4e0d\u8fc7\u4f60\u8981\u67e5\u770b\u4f60\u53bb\u7684\u65f6\u5019\u4f1a\u4e0d\u4f1a\u649e\u5230\u4ed6maintenance\u7684\u65f6\u95f4\u3002  \u6211\u4e00\u76f4\u90fd\u5728pantai cenang\u90a3\uff0ckuah town\u6211\u5c31\u6ca1\u53bb\u8fc7\uff0c\u90a3\u91cc\u5e94\u8be5\u4e5f\u662f\u6709\u4e1c\u897f\u8d70\u7684\uff08\u8001\u9e70\u5e7f\u573a\uff09  \u5bf9\u4e86\uff0c\u4f60\u7684\u9152\u5e97\u5728\u54ea\u7684\uff1f\u5c31\u770b\u770b\u9644\u8fd1\u90a3\u6709\u4ec0\u4e48\u505a\u5c31\u597d\u4e86~  \u6211\u548c\u7537\u53cb\u7684\u65c5\u884c\u4e00\u76f4\u90fd\u662f\u5728hea

In [24]:
with open('train.json', 'a') as fopen_jsonl:
    with open('/home/husein/ssd3/google-translate-malay-news/translation-manglish.jsonl') as fopen:
        for l in fopen:
            try:
                l = json.loads(l)
                en = l['translation'].get('standard_en', '')
                ms = l['translation'].get('standard_ms', '')
                left = l['text']

                if len(en) and len(en.split()) < 2000 and len(left) and len(left.split()) < 2000:
                    s = f'Manglish to standard English: {left} {en}'
                    
                    fopen_jsonl.write(f'{json.dumps(s)}\n')
                    fopen_jsonl.flush()

                if found_word(ms, rejected_words):
                    continue

                if len(ms) and len(ms.split()) < 2000 and len(left) and len(left.split()) < 2000:
                    s = f'Manglish to standard Malay: {left} {ms}'
                    
                    fopen_jsonl.write(f'{json.dumps(s)}\n')
                    fopen_jsonl.flush()
                
            except:
                pass

In [25]:
with open('train.json', 'a') as fopen_jsonl:
    with open('/home/husein/ssd3/google-translate-malay-news/translation-c.cari.com.my.jsonl') as fopen:
        for l in fopen:
            try:
                l = json.loads(l)
                en = l['translation'].get('standard_en', '')
                ms = l['translation'].get('standard_ms', '')
                left = l['text']

                if len(en) and len(en.split()) < 2000 and len(left) and len(left.split()) < 2000:
                    s = f'social media Mandarin to standard English: {left} {en}'
                    
                    fopen_jsonl.write(f'{json.dumps(s)}\n')
                    fopen_jsonl.flush()


                if found_word(ms, rejected_words):
                    continue

                if len(ms) and len(ms.split()) < 2000 and len(left) and len(left.split()) < 2000:
                    s = f'social media Mandarin to standard Malay: {left} {ms}'
                    
                    fopen_jsonl.write(f'{json.dumps(s)}\n')
                    fopen_jsonl.flush()
                    
            except:
                pass

In [26]:
with open('train.json', 'a') as fopen_jsonl:
    with open('/home/husein/ssd3/google-translate-ms-zh-CN/wiki-jawi-ms-en.jsonl') as fopen:
        for l in fopen:
            l = json.loads(l)
            en = l['en']
            ms = l['malay']
            left = l['jawi']

            if len(en) and len(en.split()) < 2000 and len(left) and len(left.split()) < 2000:
                
                s = f'Standard English to Jawi: {en} {left}'
                    
                fopen_jsonl.write(f'{json.dumps(s)}\n')
                fopen_jsonl.flush()
                
            if len(ms) and len(ms.split()) < 2000 and len(left) and len(left.split()) < 2000:
                
                s = f'Standard Malay to Jawi: {ms} {left}'
                    
                fopen_jsonl.write(f'{json.dumps(s)}\n')
                fopen_jsonl.flush()

In [27]:
!tail -n 1 train.json

"Standard Malay to Jawi: Wilayah Plzen (bahasa Czech Plzensky kraj ) merupakan sebuah daerah di Czech yang memiliki keluasan wilayah 7,561 km2 dan populasi 551,528 orang pada (2006). Ibu kotanya ialah Plzen. Pembahagian pentadbiran. Komun. Sejak 1 Januari 2003, wilayah ini terbahagi kepada 15 Kawasan perbandaran \u0648\u0644\u0627\u064a\u0647 \u06a4\u0644\u0632\u064a\u0646 \ufd3f\u0628\u0647\u0627\u0633 \u0686\u0632\u064a\u0686\u0647 \u06a4\u0644\u0632\u064a\u0646\u0633\u0643\u0627\u064a \u0643\u0631\u0627\u062c \ufd3e \u0645\u0631\u0648\u06a4\u0627\u0643\u0646 \u0633\u0628\u0648\u0627\u0647 \u062f\u0627\u064a\u0631\u0647 \u062f \u0686\u0632\u064a\u0686\u0647 \u064a\u06a0 \u0645\u0645\u064a\u0644\u064a\u0642\u064a \u0643\u0644\u0648\u0627\u0633\u0646 \u0648\u0644\u0627\u064a\u0647 \u0667\u060c\u0665\u0666\u0661 \u0642\u0645\u0662 \u062f\u0627\u0646 \u06a4\u0648\u06a4\u0648\u0644\u0633\u064a \u0665\u0665\u0661\u060c\u0665\u0662\u0668 \u0627\u0648\u0631\u06a0 \u06a4\u062f \ufd3f\u0662\u0

In [28]:
import random

In [29]:
with open('train.json', 'a') as fopen_jsonl:
    with open('/home/husein/ssd3/google-translate-malay-news/synthetic-word-switching-translation.jsonl') as fopen:
        for l in fopen:
            l = json.loads(l)
            ms = l['ms']
            en = l['en']
            if not len(l['augmentation']):
                continue
            augmentation = random.sample(l['augmentation'], min(len(l['augmentation']), 2))
            for a in augmentation:
                
                s = f'to English: {a} {en}'
                    
                fopen_jsonl.write(f'{json.dumps(s)}\n')
                fopen_jsonl.flush()
                
                s = f'to Malay: {a} {ms}'
                    
                fopen_jsonl.write(f'{json.dumps(s)}\n')
                fopen_jsonl.flush()

In [30]:
!tail -n 1 train.json

"to Malay: Tok expect \u06a4\u0648\u0644\u0627 filem pertama kaih mendapat \u0644\u0627\u06a4\u0646 \u06a4\u0646\u0686\u0644\u0648\u0646\u0646 dalam FFM28. Tidak sangka pula filem pertama saya mendapat lapan pencalonan dalam FFM28."


In [31]:
!ls -lha train.json

-rw-rw-r-- 1 husein husein 11G Ogos  5 16:25 train.json


In [ ]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="train.json",
    path_in_repo="pretrain-translation.jsonl",
    repo_id="malaysia-ai/pretrain-text-dataset",
    repo_type="dataset",
)


train.json:   0%|          | 0.00/11.4G [00:00<?, ?B/s]